In [1]:
!pip cache purge
!pip uninstall -y torch torchtext fastai timm torchvision torchaudio torchdata
!pip install torch==2.3.0 torchtext==0.18.0

Files removed: 0
Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: fastai 2.7.18
Uninstalling fastai-2.7.18:
  Successfully uninstalled fastai-2.7.18
Found existing installation: timm 1.0.11
Uninstalling timm-1.0.11:
  Successfully uninstalled timm-1.0.11
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


https://www.kaggle.com/code/gmhost/gru-capsule

Sentiment analysis based on improved pre-trained word embeddings
https://www.sciencedirect.com/science/article/abs/pii/S095741741830558X

In [3]:
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict,Counter
import re
import numpy as np
import spacy
import pickle
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, SubsetRandomSampler
from sklearn.feature_extraction.text import CountVectorizer
from torchtext.vocab import GloVe
from sklearn.metrics import accuracy_score, f1_score

/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


### **Custom Word Positivity Score:**

In [ ]:
import spacy
import re
from collections import defaultdict
import pandas as pd

nlp = spacy.load("en_core_web_sm")
label_map = {-1: 'negative', 0: 'neutral', 1: 'positive'}
stopwords = {"a", "the", "and", "is", "in", "to", "for", "of", "on", "it", "this", "that", "with", "as", "at", "by", "an"}
min_frequency = 10
relevant_pos_tags = {"ADJ", "ADV", "VERB", "NOUN"}

word_counts = defaultdict(lambda: {'positive': 0, 'neutral': 0, 'negative': 0})
word_frequencies = defaultdict(int)

df = pd.read_csv("/content/train.csv")

sentences = df['sentence'].str.lower().tolist()
labels = df['gold_label'].tolist()

for doc, label in zip(nlp.pipe(sentences, batch_size=32), labels):
    sentiment = label_map.get(label, 'neutral')
    for token in doc:
        if token.is_alpha and token.pos_ in relevant_pos_tags and token.text not in stopwords:
            word_counts[token.text][sentiment] += 1
            word_frequencies[token.text] += 1

word_scores = {}
for word, counts in word_counts.items():
    if word_frequencies[word] >= min_frequency:
        total_count = counts['positive'] + counts['neutral'] + counts['negative']
        if total_count > 0:
            score = (counts['positive'] - counts['negative']) / total_count
            word_scores[word] = score

word_scores_df = pd.DataFrame(list(word_scores.items()), columns=['word', 'positivity_score'])
word_scores_df = word_scores_df.sort_values(by='positivity_score', ascending=False)
output_path = 'word_positivity_scores.csv'
word_scores_df.to_csv(output_path, index=False)
print(f"Word positivity scores saved to {output_path}")


###**GRU+ Glove + Custom Positivity score + POS :**

In [4]:
nlp = spacy.load("en_core_web_sm")
glove = GloVe(name='6B', dim=100)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

.vector_cache/glove.6B.zip: 862MB [02:38, 5.43MB/s]                           
100%|█████████▉| 399999/400000 [00:20<00:00, 19080.04it/s]


Using device: cuda


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

positivity_scores_df = pd.read_csv("/content/drive/MyDrive/NLP/Sentiment_Analysis_Competition/word_positivity_scores.csv")
positivity_scores = dict(zip(positivity_scores_df['word'], positivity_scores_df['positivity_score']))

pos_tags = ["NOUN", "VERB", "ADJ", "ADV", "PRON", "DET", "ADP", "NUM", "CONJ", "PART", "INTJ", "AUX"]
pos_embedding_layer = nn.Embedding(len(pos_tags), 30)
pos_to_idx = {tag: idx for idx, tag in enumerate(pos_tags)}

dep_tags = list(nlp.get_pipe("parser").labels)
dep_embedding_layer = nn.Embedding(len(dep_tags), 30)
dep_to_idx = {dep: idx for idx, dep in enumerate(dep_tags)}

negation_words = {
    "not", "no", "never", "n't", "cannot", "can't", "won't",
    "didn't", "doesn't", "isn't", "aren't", "wasn't", "weren't",
    "shouldn't", "wouldn't", "couldn't", "mustn't", "mightn't",
    "needn't", "without", "nor", "neither", "nowhere", "none", "nothing", "nobody", "hardly", "scarcely", "barely"
}

def get_dependency_contextual_embedding(tokens, index):
    token = tokens[index]
    dependent_tokens = [child for child in token.children]
    context_embeddings = []
    for dep_token in dependent_tokens:
        lower_word = dep_token.text.lower()
        embedding = glove[lower_word] if lower_word in glove.stoi else torch.zeros(100)
        context_embeddings.append(embedding)
    if context_embeddings:
        return torch.mean(torch.stack(context_embeddings), dim=0)
    return torch.zeros(100)


def get_token_embedding(token, tokens):
    original_word = token.text
    lower_word = original_word.lower()
    if lower_word in glove.stoi:
        embedding = glove[lower_word]
        if torch.sum(embedding) == 0.0:
            embedding = torch.randn(100)
    else:
        embedding = torch.randn(100)

    # POS Embedding
    pos_index = pos_to_idx.get(token.pos_, len(pos_tags) - 1)
    pos_embedding = pos_embedding_layer(torch.tensor(pos_index)).detach()

    # Positivity Embedding
    positivity_score = 5 * positivity_scores.get(lower_word, 0.0)
    positivity_embedding = torch.tensor([positivity_score], dtype=torch.float32)

    # Dependency Embedding
    dep_index = dep_to_idx.get(token.dep_, len(dep_tags) - 1)
    dep_embedding = dep_embedding_layer(torch.tensor(dep_index)).detach()

    # Negation Feature
    negation = 1.0 if any(ancestor.text.lower() in negation_words for ancestor in token.ancestors) else 0.0
    negation_embedding = torch.tensor([negation], dtype=torch.float32)

    # Contextual Embedding
    contextual_embedding = get_dependency_contextual_embedding(tokens, token.i) if len(tokens) > 1 else torch.zeros(100)

    # Punctuation Feature
    punctuation = 1.0 if any(punct in token.sent.text for punct in {"!", "?"}) else 0.0
    punctuation_embedding = torch.tensor([punctuation], dtype=torch.float32)

    # Capitalization Feature
    capitalization = 1.0 if original_word.isupper() else 0.0
    capitalization_embedding = torch.tensor([capitalization], dtype=torch.float32)

    return torch.cat((
        embedding,
        contextual_embedding,
        pos_embedding,
        dep_embedding,
        positivity_embedding,
        negation_embedding,
        punctuation_embedding,
        capitalization_embedding
    ))

df = pd.read_csv("/content/drive/MyDrive/NLP/Sentiment_Analysis_Competition/Data/train.csv")
df['gold_label'] = df['gold_label'].map({-1: 0, 0: 1, 1: 2})


MAX_TOKENS = 64
from tqdm import tqdm

precomputed_data = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing Sentences"):
    text = row['sentence']
    label = row['gold_label']
    doc = nlp(text)
    limited_tokens = list(doc)[:MAX_TOKENS]
    embeddings = [get_token_embedding(token, doc) for token in limited_tokens]
    precomputed_data.append((torch.stack(embeddings), label))


Processing Sentences: 100%|██████████| 92228/92228 [18:55<00:00, 81.22it/s]


**TextCNN:**

In [22]:
labels = [item[1] for item in precomputed_data]
train_data, val_data = train_test_split(precomputed_data, test_size=0.1, stratify=labels, random_state=42)
import torch.nn.functional as F

class SentimentDatasetWrapper(Dataset):
    def __init__(self, embeddings, labels=None):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        if self.labels is not None:
            return self.embeddings[idx], self.labels[idx]
        return self.embeddings[idx]

class RCNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.5):
        super(RCNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.conv = nn.Conv1d(in_channels=hidden_dim * 2, out_channels=hidden_dim, kernel_size=3, padding=1)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out.permute(0, 2, 1)
        conv_out = torch.relu(self.conv(lstm_out))
        pooled = torch.max(conv_out, dim=2)[0]
        logits = self.fc(self.dropout(pooled))
        return logits

# Model Initialization
model = RCNN(
    input_dim=264,
    hidden_dim=512,
    output_dim=3,
    dropout_prob=0.5
).to(device)


# Training and Validation
def train_and_validate(train_data, val_data, model, epochs=40, batch_size=64, patience_threshold = 5):
    training_dataset = SentimentDatasetWrapper([x[0] for x in train_data], [x[1] for x in train_data])
    validation_dataset = SentimentDatasetWrapper([x[0] for x in val_data], [x[1] for x in val_data])

    training_data_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, collate_fn=lambda batch: zip(*batch))
    validation_data_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda batch: zip(*batch))

    loss_function = nn.CrossEntropyLoss()
    optimizer_instance = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    learning_rate_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_instance, mode='min', patience=5, verbose=True)

    highest_validation_f1 = 0
    early_stopping_counter = 0

    for epoch in range(epochs):
        model.train()
        training_losses = []
        ground_truth_train = []
        predicted_train = []

        for batch_inputs, batch_labels in training_data_loader:
            padded_inputs = torch.nn.utils.rnn.pad_sequence(batch_inputs, batch_first=True).to(device)
            tensor_labels = torch.tensor(batch_labels).to(device)

            optimizer_instance.zero_grad()
            model_outputs = model(padded_inputs)
            loss_value = loss_function(model_outputs, tensor_labels)
            loss_value.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer_instance.step()

            training_losses.append(loss_value.item())
            _, predictions = torch.max(model_outputs, 1)
            ground_truth_train.extend(tensor_labels.cpu().numpy())
            predicted_train.extend(predictions.cpu().numpy())

        training_accuracy = accuracy_score(ground_truth_train, predicted_train) * 100
        training_f1_score = f1_score(ground_truth_train, predicted_train, average='weighted') * 100
        average_training_loss = np.mean(training_losses)

        model.eval()
        validation_losses = []
        ground_truth_val = []
        predicted_val = []

        with torch.no_grad():
            for batch_inputs, batch_labels in validation_data_loader:
                padded_inputs = torch.nn.utils.rnn.pad_sequence(batch_inputs, batch_first=True).to(device)
                tensor_labels = torch.tensor(batch_labels).to(device)

                model_outputs = model(padded_inputs)
                loss_value = loss_function(model_outputs, tensor_labels)
                validation_losses.append(loss_value.item())
                _, predictions = torch.max(model_outputs, 1)
                ground_truth_val.extend(tensor_labels.cpu().numpy())
                predicted_val.extend(predictions.cpu().numpy())

        average_validation_loss = np.mean(validation_losses)
        validation_accuracy = accuracy_score(ground_truth_val, predicted_val) * 100
        validation_f1_score = f1_score(ground_truth_val, predicted_val, average='weighted') * 100
        learning_rate_scheduler.step(average_validation_loss)

        print(f"Epoch {epoch + 1} - "
              f"Train Loss: {average_training_loss:.4f}, Train Accuracy: {training_accuracy:.2f}%, Train F1: {training_f1_score:.2f}%, "
              f"Val Loss: {average_validation_loss:.4f}, Val Accuracy: {validation_accuracy:.2f}%, Val F1: {validation_f1_score:.2f}%")

        if validation_f1_score > highest_validation_f1:
            highest_validation_f1 = validation_f1_score
            early_stopping_counter = 0
            torch.save(model.state_dict(), 'best_model_state.pt')
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience_threshold:
                print("Early stopping triggered.")
                break

    model.load_state_dict(torch.load('best_model_state.pt'))
    print(f"Best Validation F1 Score: {highest_validation_f1:.2f}%")

train_and_validate(train_data, val_data, model, epochs=40, batch_size=32, patience_threshold=5)


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1 - Train Loss: 0.7464, Train Accuracy: 68.55%, Train F1: 67.60%, Val Loss: 0.7124, Val Accuracy: 69.66%, Val F1: 69.69%
Epoch 2 - Train Loss: 0.6769, Train Accuracy: 71.70%, Train F1: 71.08%, Val Loss: 0.6773, Val Accuracy: 71.58%, Val F1: 70.61%
Epoch 3 - Train Loss: 0.6365, Train Accuracy: 73.54%, Train F1: 73.03%, Val Loss: 0.6565, Val Accuracy: 72.19%, Val F1: 71.72%
Epoch 4 - Train Loss: 0.6010, Train Accuracy: 75.10%, Train F1: 74.66%, Val Loss: 0.6615, Val Accuracy: 72.67%, Val F1: 72.00%
Epoch 5 - Train Loss: 0.5652, Train Accuracy: 76.73%, Train F1: 76.38%, Val Loss: 0.6832, Val Accuracy: 72.26%, Val F1: 71.28%
Epoch 6 - Train Loss: 0.5278, Train Accuracy: 78.39%, Train F1: 78.08%, Val Loss: 0.6996, Val Accuracy: 72.08%, Val F1: 70.97%
Epoch 7 - Train Loss: 0.4890, Train Accuracy: 80.04%, Train F1: 79.80%, Val Loss: 0.7175, Val Accuracy: 72.35%, Val F1: 72.07%
Epoch 8 - Train Loss: 0.4523, Train Accuracy: 81.58%, Train F1: 81.37%, Val Loss: 0.7618, Val Accuracy: 72.59%,

In [29]:
labels = [item[1] for item in precomputed_data]
train_data, val_data = train_test_split(precomputed_data, test_size=0.1, stratify=labels, random_state=42)
import torch.nn.functional as F

class SentimentDatasetWrapper(Dataset):
    def __init__(self, embeddings, labels=None):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        if self.labels is not None:
            return self.embeddings[idx], self.labels[idx]
        return self.embeddings[idx]

class RCNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.5):
        super(RCNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.conv = nn.Conv1d(in_channels=hidden_dim * 2, out_channels=hidden_dim, kernel_size=3, padding=1)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out.permute(0, 2, 1)
        conv_out = torch.relu(self.conv(lstm_out))
        pooled = torch.max(conv_out, dim=2)[0]
        logits = self.fc(self.dropout(pooled))
        return logits

# Model Initialization
model = RCNN(
    input_dim=264,
    hidden_dim=512,
    output_dim=3,
    dropout_prob=0.5
).to(device)

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, logits, targets):
        probs = F.softmax(logits, dim=-1)
        targets_one_hot = F.one_hot(targets, num_classes=logits.size(-1)).float()
        log_probs = F.log_softmax(logits, dim=-1)
        focal_weight = self.alpha * (1 - probs).pow(self.gamma)
        loss = -focal_weight * targets_one_hot * log_probs
        return loss.sum(dim=-1).mean()


# Training and Validation
def train_and_validate(train_data, val_data, model, epochs=40, batch_size=64, patience_threshold = 5):
    training_dataset = SentimentDatasetWrapper([x[0] for x in train_data], [x[1] for x in train_data])
    validation_dataset = SentimentDatasetWrapper([x[0] for x in val_data], [x[1] for x in val_data])

    training_data_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, collate_fn=lambda batch: zip(*batch))
    validation_data_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda batch: zip(*batch))

    loss_function = FocalLoss(alpha=1, gamma=2)
    optimizer_instance = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    learning_rate_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_instance, mode='min', patience=5, verbose=True)

    highest_validation_f1 = 0
    early_stopping_counter = 0

    for epoch in range(epochs):
        model.train()
        training_losses = []
        ground_truth_train = []
        predicted_train = []

        for batch_inputs, batch_labels in training_data_loader:
            padded_inputs = torch.nn.utils.rnn.pad_sequence(batch_inputs, batch_first=True).to(device)
            tensor_labels = torch.tensor(batch_labels).to(device)

            optimizer_instance.zero_grad()
            model_outputs = model(padded_inputs)
            loss_value = loss_function(model_outputs, tensor_labels)
            loss_value.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer_instance.step()

            training_losses.append(loss_value.item())
            _, predictions = torch.max(model_outputs, 1)
            ground_truth_train.extend(tensor_labels.cpu().numpy())
            predicted_train.extend(predictions.cpu().numpy())

        training_accuracy = accuracy_score(ground_truth_train, predicted_train) * 100
        training_f1_score = f1_score(ground_truth_train, predicted_train, average='weighted') * 100
        average_training_loss = np.mean(training_losses)

        model.eval()
        validation_losses = []
        ground_truth_val = []
        predicted_val = []

        with torch.no_grad():
            for batch_inputs, batch_labels in validation_data_loader:
                padded_inputs = torch.nn.utils.rnn.pad_sequence(batch_inputs, batch_first=True).to(device)
                tensor_labels = torch.tensor(batch_labels).to(device)

                model_outputs = model(padded_inputs)
                loss_value = loss_function(model_outputs, tensor_labels)
                validation_losses.append(loss_value.item())
                _, predictions = torch.max(model_outputs, 1)
                ground_truth_val.extend(tensor_labels.cpu().numpy())
                predicted_val.extend(predictions.cpu().numpy())

        average_validation_loss = np.mean(validation_losses)
        validation_accuracy = accuracy_score(ground_truth_val, predicted_val) * 100
        validation_f1_score = f1_score(ground_truth_val, predicted_val, average='weighted') * 100
        learning_rate_scheduler.step(average_validation_loss)

        print(f"Epoch {epoch + 1} - "
              f"Train Loss: {average_training_loss:.4f}, Train Accuracy: {training_accuracy:.2f}%, Train F1: {training_f1_score:.2f}%, "
              f"Val Loss: {average_validation_loss:.4f}, Val Accuracy: {validation_accuracy:.2f}%, Val F1: {validation_f1_score:.2f}%")

        if validation_f1_score > highest_validation_f1:
            highest_validation_f1 = validation_f1_score
            early_stopping_counter = 0
            torch.save(model.state_dict(), 'best_model_state.pt')
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience_threshold:
                print("Early stopping triggered.")
                break

    model.load_state_dict(torch.load('best_model_state.pt'))
    print(f"Best Validation F1 Score: {highest_validation_f1:.2f}%")

train_and_validate(train_data, val_data, model, epochs=40, batch_size=32, patience_threshold=5)


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1 - Train Loss: 0.3162, Train Accuracy: 67.98%, Train F1: 67.04%, Val Loss: 0.2907, Val Accuracy: 69.70%, Val F1: 68.85%
Epoch 2 - Train Loss: 0.2828, Train Accuracy: 71.14%, Train F1: 70.48%, Val Loss: 0.2844, Val Accuracy: 70.39%, Val F1: 68.96%
Epoch 3 - Train Loss: 0.2650, Train Accuracy: 72.57%, Train F1: 71.99%, Val Loss: 0.2751, Val Accuracy: 71.50%, Val F1: 71.41%
Epoch 4 - Train Loss: 0.2506, Train Accuracy: 73.69%, Train F1: 73.17%, Val Loss: 0.2727, Val Accuracy: 71.52%, Val F1: 70.66%
Epoch 5 - Train Loss: 0.2368, Train Accuracy: 74.59%, Train F1: 74.13%, Val Loss: 0.2745, Val Accuracy: 71.82%, Val F1: 71.40%
Epoch 6 - Train Loss: 0.2229, Train Accuracy: 75.54%, Train F1: 75.14%, Val Loss: 0.2825, Val Accuracy: 72.22%, Val F1: 71.74%
Epoch 7 - Train Loss: 0.2094, Train Accuracy: 76.85%, Train F1: 76.52%, Val Loss: 0.2965, Val Accuracy: 72.76%, Val F1: 72.35%
Epoch 8 - Train Loss: 0.1966, Train Accuracy: 77.85%, Train F1: 77.54%, Val Loss: 0.3158, Val Accuracy: 70.96%,

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


# Dataset Wrapper
class SentimentDatasetWrapper(Dataset):
    def __init__(self, embeddings, labels=None):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        if self.labels is not None:
            return self.embeddings[idx], self.labels[idx]
        return self.embeddings[idx]


# Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(1)]


# Attention Mechanism
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.attention_weights = nn.Linear(hidden_dim * 2, 1)

    def forward(self, lstm_out):
        scores = self.attention_weights(lstm_out).squeeze(-1)
        weights = torch.softmax(scores, dim=1)
        context = torch.sum(weights.unsqueeze(-1) * lstm_out, dim=1)
        return context


# Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=1.0, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean()


# RCNN Model
class RCNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.5):
        super(RCNN, self).__init__()
        self.hidden_dim = hidden_dim

        # Positional Embedding
        self.positional_encoding = PositionalEncoding(d_model=input_dim)

        # Linear layer to match dimensions for residual connection
        self.input_projection = nn.Linear(input_dim, hidden_dim * 2)

        # Bidirectional LSTM
        self.lstm = nn.LSTM(input_dim, hidden_dim, bidirectional=True, batch_first=True)

        # Multi-kernel Convolutions
        self.conv1 = nn.Conv1d(hidden_dim * 2, hidden_dim, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(hidden_dim * 2, hidden_dim, kernel_size=5, padding=2)

        # Layer Normalization
        self.layer_norm = nn.LayerNorm(hidden_dim * 2)

        # Attention Mechanism
        self.attention = Attention(hidden_dim)

        # Fully Connected Layers
        combined_features_dim = (hidden_dim * 2) + (hidden_dim * 2)  # Size of pooled_conv + attention_context
        self.fc = nn.Linear(combined_features_dim, output_dim)  # Adjusted input size
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        # Apply positional encoding
        x = self.positional_encoding(x)

        # Linear projection to match LSTM output dimension
        projected_input = self.input_projection(x)  # Shape: (batch_size, seq_len, hidden_dim * 2)

        # LSTM Layer with Residual Connection
        lstm_out, _ = self.lstm(x)  # Shape: (batch_size, seq_len, hidden_dim * 2)
        lstm_out = self.layer_norm(lstm_out + projected_input)  # Add residual connection and normalize

        # Convolutional Layers
        lstm_out_perm = lstm_out.permute(0, 2, 1)  # Shape: (batch_size, hidden_dim * 2, seq_len)
        conv1_out = torch.relu(self.conv1(lstm_out_perm))  # Shape: (batch_size, hidden_dim, seq_len)
        conv2_out = torch.relu(self.conv2(lstm_out_perm))  # Shape: (batch_size, hidden_dim, seq_len)
        conv_out = torch.cat([conv1_out, conv2_out], dim=1)  # Concatenate multi-kernel outputs

        # Global Max Pooling
        pooled_conv = torch.max(conv_out, dim=2)[0]  # Shape: (batch_size, hidden_dim * 2)

        # Attention Layer
        attention_context = self.attention(lstm_out)  # Shape: (batch_size, hidden_dim * 2)

        # Combine Features and Fully Connected Layer
        combined_features = torch.cat([pooled_conv, attention_context], dim=1)  # Shape: (batch_size, combined_features_dim)
        logits = self.fc(self.dropout(combined_features))  # Final output
        return logits

# Initialize Model
model = RCNN(
    input_dim=264,
    hidden_dim=512,
    output_dim=3,
    dropout_prob=0.5
).to(device)


# Training and Validation Function
def train_and_validate(train_data, val_data, model, epochs=40, batch_size=32, patience_threshold=5):
    training_dataset = SentimentDatasetWrapper([x[0] for x in train_data], [x[1] for x in train_data])
    validation_dataset = SentimentDatasetWrapper([x[0] for x in val_data], [x[1] for x in val_data])

    training_data_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, collate_fn=lambda batch: zip(*batch))
    validation_data_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda batch: zip(*batch))

    loss_function = FocalLoss(alpha=1.0, gamma=2.0)
    optimizer_instance = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    learning_rate_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_instance, mode='min', patience=5, verbose=True)

    highest_validation_f1 = 0
    early_stopping_counter = 0

    for epoch in range(epochs):
        model.train()
        training_losses = []
        ground_truth_train = []
        predicted_train = []

        for batch_inputs, batch_labels in training_data_loader:
            padded_inputs = torch.nn.utils.rnn.pad_sequence(batch_inputs, batch_first=True).to(device)
            tensor_labels = torch.tensor(batch_labels).to(device)

            optimizer_instance.zero_grad()
            model_outputs = model(padded_inputs)
            loss_value = loss_function(model_outputs, tensor_labels)
            loss_value.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer_instance.step()

            training_losses.append(loss_value.item())
            _, predictions = torch.max(model_outputs, 1)
            ground_truth_train.extend(tensor_labels.cpu().numpy())
            predicted_train.extend(predictions.cpu().numpy())

        training_accuracy = accuracy_score(ground_truth_train, predicted_train) * 100
        training_f1_score = f1_score(ground_truth_train, predicted_train, average='weighted') * 100
        average_training_loss = np.mean(training_losses)

        model.eval()
        validation_losses = []
        ground_truth_val = []
        predicted_val = []

        with torch.no_grad():
            for batch_inputs, batch_labels in validation_data_loader:
                padded_inputs = torch.nn.utils.rnn.pad_sequence(batch_inputs, batch_first=True).to(device)
                tensor_labels = torch.tensor(batch_labels).to(device)

                model_outputs = model(padded_inputs)
                loss_value = loss_function(model_outputs, tensor_labels)
                validation_losses.append(loss_value.item())
                _, predictions = torch.max(model_outputs, 1)
                ground_truth_val.extend(tensor_labels.cpu().numpy())
                predicted_val.extend(predictions.cpu().numpy())

        average_validation_loss = np.mean(validation_losses)
        validation_accuracy = accuracy_score(ground_truth_val, predicted_val) * 100
        validation_f1_score = f1_score(ground_truth_val, predicted_val, average='weighted') * 100
        learning_rate_scheduler.step(average_validation_loss)

        print(f"Epoch {epoch + 1} - "
              f"Train Loss: {average_training_loss:.4f}, Train Accuracy: {training_accuracy:.2f}%, Train F1: {training_f1_score:.2f}%, "
              f"Val Loss: {average_validation_loss:.4f}, Val Accuracy: {validation_accuracy:.2f}%, Val F1: {validation_f1_score:.2f}%")

        if validation_f1_score > highest_validation_f1:
            highest_validation_f1 = validation_f1_score
            early_stopping_counter = 0
            torch.save(model.state_dict(), 'best_model_state.pt')
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience_threshold:
                print("Early stopping triggered.")
                break

    model.load_state_dict(torch.load('best_model_state.pt'))
    print(f"Best Validation F1 Score: {highest_validation_f1:.2f}%")

train_and_validate(train_data, val_data, model, epochs=40, batch_size=32, patience_threshold=5)


Epoch 1 - Train Loss: 0.3860, Train Accuracy: 62.12%, Train F1: 60.83%, Val Loss: 0.3323, Val Accuracy: 64.04%, Val F1: 60.50%
Epoch 2 - Train Loss: 0.3262, Train Accuracy: 66.77%, Train F1: 65.69%, Val Loss: 0.3630, Val Accuracy: 63.45%, Val F1: 61.95%
Epoch 3 - Train Loss: 0.3185, Train Accuracy: 67.67%, Train F1: 66.60%, Val Loss: 0.3169, Val Accuracy: 68.24%, Val F1: 66.81%
Epoch 4 - Train Loss: 0.3121, Train Accuracy: 68.41%, Train F1: 67.39%, Val Loss: 0.3151, Val Accuracy: 67.35%, Val F1: 66.70%
Epoch 5 - Train Loss: 0.3081, Train Accuracy: 68.83%, Train F1: 67.86%, Val Loss: 0.3033, Val Accuracy: 68.78%, Val F1: 66.99%
Epoch 6 - Train Loss: 0.3044, Train Accuracy: 69.18%, Train F1: 68.22%, Val Loss: 0.3050, Val Accuracy: 68.46%, Val F1: 66.57%
Epoch 7 - Train Loss: 0.3025, Train Accuracy: 69.38%, Train F1: 68.44%, Val Loss: 0.3026, Val Accuracy: 69.68%, Val F1: 67.99%
Epoch 8 - Train Loss: 0.2999, Train Accuracy: 69.56%, Train F1: 68.63%, Val Loss: 0.3024, Val Accuracy: 69.74%,

KeyboardInterrupt: 

In [30]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
from tqdm import tqdm

dev_df = pd.read_csv("/content/drive/MyDrive/NLP/Sentiment_Analysis_Competition/Data/dev.csv")
precomputed_dev_data = []

for _, row in tqdm(dev_df.iterrows(), total=len(dev_df)):
    text = row['sentence']
    doc = nlp(text)
    limited_tokens = list(doc)[:MAX_TOKENS]
    embeddings = [get_token_embedding(token, doc) for token in limited_tokens]
    precomputed_dev_data.append(torch.stack(embeddings))

dev_dataset = SentimentDatasetWrapper(precomputed_dev_data)
dev_data_loader = DataLoader(dev_dataset, batch_size=64, shuffle=False, collate_fn=lambda batch: batch)
model.load_state_dict(torch.load('best_model_state.pt'))
model.eval()
predictions = []

with torch.no_grad():
    for batch_inputs in tqdm(dev_data_loader):
        padded_inputs = torch.nn.utils.rnn.pad_sequence(batch_inputs, batch_first=True).to(device)
        outputs = model(padded_inputs)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())
adjusted_predictions = [pred - 1 for pred in predictions]
with open('answer.txt', 'w') as f:
    for pred in adjusted_predictions:
        f.write(f"{pred}\n")

print("Predictions saved to answer.txt")


100%|██████████| 76/76 [00:00<00:00, 83.01it/s]

Predictions saved to answer.txt
